In [1]:
import sys
sys.path.insert(0, '../' )

import time


In [2]:
from libgf264fft import clib_wrapper as cgf

# defining gf
from utils import gf2192 as gf

# the commit scheme
from commit import merkeltreecommit as mt

# hash for challenges
from utils import hash as H

from fri_ldt import m_fri_ldt as fri

def _dummy( *params ) : return None


In [3]:
def _log2( w ):
    assert 0 != w
    assert 0== (w&(w-1))
    r, t = 0 , 1
    while(w!=t): r , t = r+1 , t<<1
    return r

def _pad_len( n_or_m ):
    r = 1
    while( r < n_or_m ):  r = r<<1
    return r

In [4]:
from utils import randombytes as rd

#rd.randombytes(16)


from aesR1CS import aes128

#pt = _rand_bytes(16)
#sk = _rand_bytes(16)
pt = bytes( [i*16+i for i in range(16)] )
sk = bytes( [i for i in range(16)] )

ct, rd_states , rk , rk_tmp = aes128.aes128_encrypt( pt , sk )

print( sk )
print( pt )
print( rd_states , rk , rk_tmp  )
print( "ct0", ct )



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
b'_rd\x15W\xf5\xbc\x92\xf7\xbe;)\x1d\xb9\xf9\x1a\xff\x87\x96\x841\xd8jQdQQ\xfaw:\xd0\tL\x9c\x1ef\xf7q\xf0v,?\x86\x8eSM\xf2Vc\x85\xb7\x9f\xfcS\x8d\xf9\x97\xbeG\x8euG\xd6\x91\xf4\xbc\xd4T2\xe5T\xd0u\xf1\xd6\xc5\x1d\xd0;<\x98\x16\xeet\x00\xf8\x7fUk,\x04\x9c\x8eZ\xd06\xc5~\x1c\x15\x9a\x9b\xd2\x86\xf0_K\xe0\x98\xc649\xba\xa0=\xe7\xa1\xf9\xb5n\xd5Q,\xba_AM#\xe9\xf7N\xec\x020 \xf6\x1b\xf2\xcc\xf25<!\xc7' b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\xd6\xaat\xfd\xd2\xafr\xfa\xda\xa6x\xf1\xd6\xabv\xfe\xb6\x92\xcf\x0bd=\xbd\xf1\xbe\x9b\xc5\x00h0\xb3\xfe\xb6\xfftN\xd2\xc2\xc9\xbflY\x0c\xbf\x04i\xbfAG\xf7\xf7\xbc\x955>\x03\xf9l2\xbc\xfd\x05\x8d\xfd<\xaa\xa3\xe8\xa9\x9f\x9d\xebP\xf3\xafW\xad\xf6"\xaa^9\x0f}\xf7\xa6\x92\x96\xa7U=\xc1\n\xa3\x1fk\x14\xf9p\x1a\xe3_\xe2\x8cD\n\xdfMN\xa9\xc0&GC\x875\xa4\x1ce\xb9\xe0\x16\xba\xf4\xae\xbfz\xd2T\x992\xd1\xf0\x85Wh\x10\x93\xed\x9c\xb

In [5]:
from aesR1CS import aes128R1CS_z as R1CSz

import importlib
importlib.reload( R1CSz )

z = R1CSz.get_vec_z(pt,sk,64)

print( R1CSz.aes128R1CS_num_constrains , len(z) )


3656 3320


In [6]:
from aesR1CS import r1cs

MATs = r1cs.get_r1cs( R1CSz.aes128R1CS_num_constrains , len(z) , 64, 128)  # AES128

mat_a = MATs['A']
mat_b = MATs['B']
mat_c = MATs['C']


In [7]:
def mat_x_vec( mat , vec , n_row=-1) :
    if 0 > n_row : n_row = len(mat)
    r = [0]*n_row
    for i in range(n_row):
        ri = 0
        for j in range(len(vec)):
            ri ^= cgf.gf264_mul( mat[i][j] , vec[j] )
        r[i] = ri
    return r


In [8]:
print( "calculate Az, Bz, Cz" )
st = time.time()
Az = mat_x_vec( mat_a , z )
Bz = mat_x_vec( mat_b , z )
Cz = mat_x_vec( mat_c , z )
ed = time.time()
print( "time:", format(ed-st) , "secs" )




calculate Az, Bz, Cz
time: 25.58450198173523 secs


In [19]:


def process_R1CS( R1CS , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    mat_A , mat_B , mat_C , vec_z , witness_idx = R1CS
    n = len(vec_z)
    m = len(mat_A)
    pad_len = _pad_len( max(n,m) )
    inst_dim = _log2(witness_idx)
    dump( f"pad_len: {pad_len}, inst_dim: {inst_dim}" )

    dump( "padding and calcuate f_1v, f_z, f_w" )
    st = time.time()
    p_vec_z = vec_z + [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len-len(vec_z)) ]   # ZK
    #p_vec_z = vec_z + [ 0 for _ in range(pad_len-len(vec_z)) ]    # no ZK
    f_z = gf.ifft( p_vec_z , 1 , 0 );   p_vec_z.extend( gf.fft( f_z , 1 , pad_len ) )
    f_w = gf.polydiv( f_z , inst_dim )[witness_idx:] + [0]*witness_idx
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )
    dump( f"f_z : [{len(f_z)}] ...[{witness_idx-4}:{witness_idx+4}]:", f_z[witness_idx-4:witness_idx+4] , "..." )
    dump( f"f_w : (f_z - f_1v) /Z_{inst_dim} : " , f_w[:4] , "..." )
    #f_1v = cgf.ibtfy( vec_z[:witness_idx] , 0 )   # shouldn't compute it actually. do this for checking correctness only.
    #dump( f"f_1v : [{len(f_1v)}] ...[{witness_idx-4}:]:", f_1v[witness_idx-4:] )

    ## test ##
    #dump( f"f_1v: [{len(f_1v)}]", f_1v )
    #v_1v_256 = cgf.btfy( f_1v , 256 )
    #v_fw     = gf.fft( f_w , 1 , 256 )
    #for i in range( witness_idx ):
    #    cc0 = (p_vec_z[256+i] ^ v_1v_256[i])
    #    cc1 = cgf.gf264_mul( v_fw[i] , cgf.index_to_gf264((256+i)>>inst_dim) )
    #    dump( f"check {i}: ({p_vec_z[256+i]} ^ {v_1v_256[i]}) == Z_{inst_dim} x {v_fw[i]}" , cc0 == cc1 )


    # use global variable
    #dump( "calculate Az, Bz, Cz" )
    #st = time.time()
    #Az = mat_x_vec( mat_A , vec_z )
    #Bz = mat_x_vec( mat_B , vec_z )
    #Cz = mat_x_vec( mat_C , vec_z )
    #ed = time.time()
    #dump( "time:", format(ed-st) , "secs" )
    # XXX: check if Az*Bz == Cz here

    dump( "calculate f_Az, f_Bz, f_Cz" )
    st = time.time()
    v_Az    = Az + [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len-len(Az)) ]
    v_Bz    = Bz + [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len-len(Bz)) ]
    v_Cz    = Cz + [ gf.mul(v_Az[i],v_Bz[i]) for i in range(m,pad_len) ]           # !!! XXX: need to discuss here
    f_Az = gf.ifft( v_Az , 1 , 0 )
    f_Bz = gf.ifft( v_Bz , 1 , 0 )
    f_Cz = gf.ifft( v_Cz , 1 , 0 )
    v_Az.extend( gf.fft( f_Az , 1 , pad_len ) )
    v_Bz.extend( gf.fft( f_Bz , 1 , pad_len ) )
    v_Cz.extend( gf.fft( f_Bz , 1 , pad_len ) )
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )

    return f_w , p_vec_z , f_Az , f_Bz , f_Cz , v_Az , v_Bz , v_Cz


In [10]:

def row_check( v_Az , v_Bz , v_Cz , pad_len , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    ## row check
    dump( f"row check: calculate (f_Azxf_Bz - f_Cz) / Z_{pad_len}" )
    st = time.time()
    v_AzxBz = v_Cz[:pad_len] + [ gf.mul(v_Az[i],v_Bz[i]) for i in range(pad_len,2*pad_len) ]
    f_Azxf_Bz = gf.ifft( v_AzxBz , 1 , 0 )
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )
    dump( f"f_Azxf_Bz: [{len(f_Azxf_Bz)}] ...[{pad_len-4}:{pad_len+4}]" , f_Azxf_Bz[pad_len-4:pad_len+4] , "..." )
    #dump( f"f_Cz     : [{len(f_Cz)}] ...[{pad_len-4}:]" , f_Cz[pad_len-4:] )
    f_rowcheck = f_Azxf_Bz[pad_len:]
    dump( f"f_AB-C / Zh: [{len(f_rowcheck)}] " , f_rowcheck[:4] , "..." )
    return f_rowcheck


In [11]:

alpha = gf.from_bytes( H.gen( b'123' , bytes([1,1]) )[:gf.GF_BSIZE] )
pad_len = 4096
v_alpha =  [ 1 , alpha ] + [0]*(pad_len-2)
m = len(mat_a)
n = len(mat_a[0])
for i in range(2,m): v_alpha[i] = gf.mul( v_alpha[i-1], v_alpha[1] )
print( "evaluate values of p2A, p2B, p2C" )
st = time.time()
v_p2A , v_p2B , v_p2C = [0]*pad_len , [0]*pad_len , [0]*pad_len
for j in range(n):
        aj, bj, cj = mat_a[0][j] , mat_b[0][j] , mat_c[0][j]
        for i in range(1,m):
            aj ^= gf.mul_gf264( v_alpha[i] , mat_a[i][j] )
            bj ^= gf.mul_gf264( v_alpha[i] , mat_b[i][j] )
            cj ^= gf.mul_gf264( v_alpha[i] , mat_c[i][j] )
        v_p2A[j] , v_p2B[j] , v_p2C[j] = aj , bj , cj
ed = time.time()
print( "time:" , format(ed-st) , "secs" )



evaluate values of p2A, p2B, p2C
time: 94.4618170261383 secs


In [12]:


def lin_check( alpha , s1 , s2 , s3 , p_vec_z , v_Az , v_Bz , v_Cz , mat_A , mat_B , mat_C , pad_len , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    m = len(mat_A)
    n = len(mat_A[0])

    ## lincheck
    dump( "lin-check starts. calculate p_alpha, v_alpha" )
    st = time.time()
    v_alpha =  [ 1 , alpha ] + [0]*(pad_len-2)
    for i in range(2,m): v_alpha[i] = gf.mul( v_alpha[i-1], v_alpha[1] )
    p_alpha = gf.ifft( v_alpha , 1 , 0 )
    v_alpha.extend( gf.fft(p_alpha,1,pad_len) )
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )

    dump( "lin-check step 1. calculate p2A, p2B, p2C and evaluate their values" )
    dump( "evaluate values of p2A, p2B, p2C" )
    dump( "use global veriables" )
    #st = time.time()
    #v_p2A , v_p2B , v_p2C = [0]*pad_len , [0]*pad_len , [0]*pad_len
    #for j in range(n):
    #    aj, bj, cj = mat_A[0][j] , mat_B[0][j] , mat_C[0][j]
    #    for i in range(1,m):
    #        aj ^= gf.mul_gf264( v_alpha[i] , mat_A[i][j] )
    #        bj ^= gf.mul_gf264( v_alpha[i] , mat_B[i][j] )
    #        cj ^= gf.mul_gf264( v_alpha[i] , mat_C[i][j] )
    #    v_p2A[j] , v_p2B[j] , v_p2C[j] = aj , bj , cj
    #ed = time.time()
    #dump( "time:" , format(ed-st) , "secs" )
    dump( "interpolate p2A, p2B, p2C" )
    st = time.time()
    p2A , p2B , p2C = gf.ifft( v_p2A , 1 , 0 ) , gf.ifft( v_p2B , 1 , 0 ) , gf.ifft( v_p2C , 1 , 0 )
    v_p2A.extend( gf.fft(p2A,1,pad_len) )
    v_p2B.extend( gf.fft(p2B,1,pad_len) )
    v_p2C.extend( gf.fft(p2C,1,pad_len) )
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )

    dump( f"lin-check step 2. poly muls and /Z_{pad_len}" )
    st = time.time()
    v_sA = [ gf.mul(v_Az[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2A[i]) for i in range(2*pad_len) ]
    v_sB = [ gf.mul(v_Bz[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2B[i]) for i in range(2*pad_len) ]
    v_sC = [ gf.mul(v_Cz[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2C[i]) for i in range(2*pad_len) ]
    f_sA = gf.ifft( v_sA , 1 , 0 )
    f_sB = gf.ifft( v_sB , 1 , 0 )
    f_sC = gf.ifft( v_sC , 1 , 0 )
    g = [ gf.mul(s1,f_sA[i])^gf.mul(s2,f_sB[i])^gf.mul(s3,f_sC[i]) for i in range(pad_len) ]
    h = [ gf.mul(s1,f_sA[i])^gf.mul(s2,f_sB[i])^gf.mul(s3,f_sC[i]) for i in range(pad_len,2*pad_len) ]
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )
    dump( f"g: [{pad_len}] ...[{pad_len-4}:{pad_len}]", g[pad_len-4:] )
    dump( f"h: [{pad_len}] ...[{pad_len-4}:{pad_len}]", h[pad_len-4:] )

    return g , h

In [31]:
def commit_polys( polys , max_poly_len , RS_rho = 8 , offset = (1<<63) , verbose = 1 ):
    if 1 == verbose : dump = print
    else : dump = _dummy

    dump( "commit polys: (poly_len):" , [ len(p) for p in polys ] )

    mesg0 = []
    for pp in polys :
        v_p0 = gf.fft( pp , RS_rho * (max_poly_len//len(pp)) , offset )
        v_p1 = [ int.to_bytes(x,gf.GF_BSIZE,'little') for x in v_p0 ]
        v_p2 = [ b''.join( (v_p1[2*i] , v_p1[2*i+1]) ) for i in range(len(v_p1)//2) ]
        mesg0.append( v_p2 )
    dump( f"mesg0: ({max_poly_len}x{RS_rho}/2 = {max_poly_len*RS_rho//2}) " , [len(v) for v in mesg0] )
    mesgs = [ b''.join(e) for e in zip(*mesg0) ]
    dump( f"|mesgs:| = {len(mesgs)} , |mesgs[0]| = {len(mesgs[0])} == {gf.GF_BSIZE * 2 * len(mesg0)}" )
    rt , r_leaf , mktree = mt.commit( mesgs )
    return rt , mesgs , r_leaf , mktree


In [30]:

def generate_proof( R1CS , h_state , RS_rho = 8 , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    ## process R1CS
    mat_A , mat_B , mat_C , vec_z , witness_idx = R1CS
    n = len(vec_z)
    m = len(mat_A)
    pad_len = _pad_len( max(n,m) )
    dump( f"m(#rows): {m} x n: {n}, witness_idx: {witness_idx}, pad_len: {pad_len}" )

    f_w , p_vec_z , f_Az , f_Bz , f_Cz , v_Az , v_Bz , v_Cz = process_R1CS( R1CS , verbose )

    v_lincheck = [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len*2) ]
    for i in range(pad_len): v_lincheck[0] ^= v_lincheck[i]
    r_lincheck = gf.ifft(v_lincheck, 1 , 0 )

    r_ldt = [ gf.from_bytes(rd.randombytes(gf.GF_BSIZE)) for _ in range(pad_len*2) ]

    ## calculate RS code and commit them
    dump( "commit f_w, f_Az, f_Bz , f_Cz, r_lincheck , r_ldt" )
    offset = 1<<63
    rt0 , mesgs0 , r_leaf0 , mktree0 = commit_polys( [f_w , f_Az , f_Bz , f_Cz , r_lincheck , r_ldt ] , 2*pad_len , RS_rho , offset , verbose )
    #h_state = H.gen( h_state , rt0 )  # is is necessray theoratically ?

    ## lin check
    alpha = gf.from_bytes( H.gen( h_state , bytes([1,1]) )[:gf.GF_BSIZE] )
    s1    = gf.from_bytes( H.gen( h_state , bytes([1,2]) )[:gf.GF_BSIZE] )
    s2    = gf.from_bytes( H.gen( h_state , bytes([1,3]) )[:gf.GF_BSIZE] )
    s3    = gf.from_bytes( H.gen( h_state , bytes([1,4]) )[:gf.GF_BSIZE] )
    g , h = lin_check( alpha , s1 , s2 , s3 , p_vec_z , v_Az , v_Bz , v_Cz , mat_A , mat_B , mat_C , pad_len , verbose )

    dump( f"commit h" )
    rt1 , mesgs1 , r_leaf1 , mktree1 = commit_polys( [ h ] , 2*pad_len , RS_rho , offset , verbose )

    ## row check
    f_rowcheck = row_check(v_Az, v_Bz, v_Cz , pad_len , verbose )


    ## generate f0 for fri_ldt and perform fri_ldt
    y = [ gf.from_bytes( H.gen( h_state , bytes([2,i]) )[:gf.GF_BSIZE] ) for i in range(1,10) ]
    g_raise = list(g)
    g_raise = [0] + gf.ipolydiv(g_raise,0)[1:-1]  # raise degree by 1. It still has to be raised by pad_len.
    f0 = [ gf.mul(y[0],f_w[i])^gf.mul(y[1],f_Az[i])^gf.mul(y[2],f_Bz[i])^gf.mul(y[3],f_Cz[i])^gf.mul(y[4],f_rowcheck[i])
          ^gf.mul(y[5],r_lincheck[i])^gf.mul(y[6],h[i])^gf.mul(y[7],g[i])^gf.mul(y[9],r_ldt[i]) for i in range(pad_len) ] + [ 
              gf.mul(y[5],r_lincheck[pad_len+i])^gf.mul(y[8],gi)^gf.mul(y[9],r_ldt[pad_len+i]) for i,gi in enumerate(g_raise) ]

    ## LDT f0
    rt2 , mesgs2 , r_leaf2 , mktree2 = commit_polys( [f0] , 2*pad_len , RS_rho , offset , verbose )


    ## open something here

    return True



##### test ##########

generate_proof( (mat_a,mat_b,mat_c,z,64) , b'123' )


m(#rows): 3656 x n: 3320, witness_idx: 64, pad_len: 4096
pad_len: 4096, inst_dim: 6
padding and calcuate f_1v, f_z, f_w
time: 0.036829471588134766 secs
f_z : [4096] ...[60:68]: [11876712511685630227, 1490032541726768366, 332458469872254918, 200, 18174738647891815068, 6001071716487184560, 13518469962322655972, 233317187178767899] ...
f_w : (f_z - f_1v) /Z_6 :  [6127172004532836007169689847203516811642825044711553584273, 5331371190725866033451423644951656754162376537883345791931, 634132575255127723144808773993309673344177862854373342407, 3702310091553346639099338548455999965509975731514622240944] ...
calculate f_Az, f_Bz, f_Cz
time: 0.06415748596191406 secs
commit f_w, f_Az, f_Bz , f_Cz, r_lincheck , r_ldt
commit polys: (poly_len): [4096, 4096, 4096, 4096, 8192, 8192]
mesg0: (8192x8/2 = 32768)  [32768, 32768, 32768, 32768, 32768, 32768]
|mesgs| = 32768 , |mesgs[0| = 288 == 288
lin-check starts. calculate p_alpha, v_alpha
time: 0.06684684753417969 secs
lin-check step 1. calculate p2A, p2B

True